In [1]:
!pwd

/home/jovyan/work


In [18]:
!pip install -q -r requirements.txt

In [1]:
import ollama
import chromadb
from llama_index.core import SimpleDirectoryReader

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [14]:
MODEL = "phi:latest"

documents = SimpleDirectoryReader("/home/jovyan/work/data/tmp").load_data()

In [15]:
client = chromadb.HttpClient(host="chromadb-vecdb", port=8000)

In [6]:
# dir(client)

In [7]:
client.list_collections()[0].name

'docs'

In [8]:
# client.delete_collection??

In [8]:
collection = client.get_or_create_collection(name="docs")

In [9]:
# collection.delete?

In [10]:
# client.delete_collection(name='docs')

In [11]:
ollama_client = ollama.Client(host='http://ollama:11434')

In [10]:
ollama_chat_resp = ollama_client.chat(
    messages=[
        {
            'role': 'user',
            'content': f"Explain the concept of black hole!",
        }
    ],
    model='phi:latest',
    stream=False
)

In [11]:
ollama_chat_resp['message']['content']

" I'd be happy to help you understand the concept of black holes. \n\na black hole is a region in space where gravity is so strong that nothing, not even light, can escape from it. this includes stars, gas, and other objects that get too close to its event horizon - the boundary beyond which nothing can escape. \n\nwhen an object falls into a black hole's event horizon, it's pulled by the gravitational force towards the center of the black hole. as more matter is added, the black hole continues to grow in size, until it becomes extremely dense and has an incredibly strong gravitational pull. \n\nthis intense gravity causes time to slow down around the black hole, and creates a phenomenon known as time dilation. this means that time passes at different rates for objects closer to the black hole compared to those further away from it. \n\nthe concept of a black hole has been around since ancient times, but wasn't fully understood until the 20th century when scientists like hurwitz, berke

In [87]:
stream_resp = ollama_client.chat(
    model='phi:latest',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

In [83]:
t = ['ravi', 'choudhary']
t[-1] = 'kumar'
t

['ravi', 'kumar']

In [88]:
#test_txt = []
full_response = ""
for chunk in stream_resp:
    msg = chunk['message']['content']
    # full_response.append(msg)
    full_response += msg
    print(msg, end='', flush=True)
    #test_txt.append(chunk['message']['content'])
    #print(test_txt)
    #print(chunk['message']['content'], end='', flush=True)

 As an AI language model, I can tell you that the sky appears blue due to the phenomenon called Rayleigh scattering, which occurs when sunlight interacts with molecules in Earth's atmosphere. Specifically, blue light is scattered more than other colors because it has a shorter wavelength and higher frequency, causing it to be reflected back to our eyes from all directions. This creates the perception of a blue sky.


In [49]:
# del stream_resp

In [89]:
full_response

" As an AI language model, I can tell you that the sky appears blue due to the phenomenon called Rayleigh scattering, which occurs when sunlight interacts with molecules in Earth's atmosphere. Specifically, blue light is scattered more than other colors because it has a shorter wavelength and higher frequency, causing it to be reflected back to our eyes from all directions. This creates the perception of a blue sky.\n"

In [90]:
# ''.join(full_response)

In [11]:
documents[0].metadata

{'page_label': '1',
 'file_name': 'agnipath_defence_poilicy.pdf',
 'file_path': '/home/jovyan/work/data/tmp/agnipath_defence_poilicy.pdf',
 'file_type': 'application/pdf',
 'file_size': 131092,
 'creation_date': '2024-09-24',
 'last_modified_date': '2024-09-24'}

In [36]:
# dir(collection)

In [13]:
len(documents)

24

In [22]:
# store each document in a vector embedding database
for i, doc in enumerate(documents):
  # get the document id and text
  doc_id = doc.id_
  doc_text = doc.get_text()
  doc_metadata = doc.metadata
  response = ollama_client.embeddings(model=MODEL, prompt=doc_text)
  embedding = response["embedding"]
  collection.add(
    ids=[doc_id],
    embeddings=[embedding],
    documents=[doc_text],
    metadatas=doc_metadata
  )

In [59]:
# client.get_collection?

In [27]:
col_load = client.get_collection(name="docs")

In [61]:
prompt = "what is agnipath scheme? What are the benefits and for whom?"

In [62]:
# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama_client.embeddings(
  prompt=prompt,
  model=MODEL
)

In [63]:
results = col_load.query(
  query_embeddings=[response["embedding"]],
  n_results=2
)

In [64]:
results

{'ids': [['8e0b53c5-f68d-45e4-92a6-f61f84e5d241',
   'e4005970-9a8f-4be2-b97c-ff6049cf3f16']],
 'distances': [[4737.008870330587, 5455.332635399611]],
 'embeddings': None,
 'metadatas': [[{'creation_date': '2024-09-24',
    'file_name': 'agnipath_defence_poilicy.pdf',
    'file_path': '/home/jovyan/work/data/tmp/agnipath_defence_poilicy.pdf',
    'file_size': 131092,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-09-24',
    'page_label': '4'},
   {'creation_date': '2024-09-24',
    'file_name': 'agnipath_defence_poilicy.pdf',
    'file_path': '/home/jovyan/work/data/tmp/agnipath_defence_poilicy.pdf',
    'file_size': 131092,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-09-24',
    'page_label': '2'}]],
 'documents': [[' \n be enrolled in regular cadre of the Armed Forces. Detailed guidelines will be \nissued separately.  \n \n Enrolment will be undertaken through an online centralised system for \nall three services with specialised rallies 

In [30]:
results['documents'][0]

[' \n be enrolled in regular cadre of the Armed Forces. Detailed guidelines will be \nissued separately.  \n \n Enrolment will be undertaken through an online centralised system for \nall three services with specialised rallies and  campus interviews from \nrecognised technical institutes such as Industrial Training Institutes and \nNational Skills Qualifications Framework,  among others.  Enrolment will be \nbased on ‘All India All Class’ basis and the eligible age will be in range from \n17.5 to 21 years. Agniveers  will meet the medical eligibility conditions laid \ndown for enrolment in the armed forces as applicable to respective \ncategories/trades. The educational qualification for Agniveers will remain as \nin vogue for enrollment in various categories. {For example: For entry into \nGeneral Duty (GD) soldier, the educational qualification is Class 10}.  \n \n \n \n \n ',
 ' \n scheme.  The national stands to immensely benefit by infusion of highly \ninspired youth with deeper 

In [31]:
data = " ".join(results['documents'][0])

In [33]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama_client.generate(
  model=MODEL,
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

 The Agnipath Scheme is a new recruitment policy implemented by the Government of India that aims to recruit young individuals into the three services, namely the Army, Navy and Air Force, through an online centralised system. Enrolment will be undertaken based on 'All-India All Class' basis and eligible candidates must meet the medical eligibility conditions laid down for enrolment in different categories or trades. The educational qualification for Agniveers remains unchanged for enrollment into various categories/trades such as General Duty soldier, where educational qualifications should not be less than Class 10. 
Agnipath Scheme offers a customised monthly package with Risk and Hardship allowances, which are to be paid upon completion of four years of service in the Armed Forces. Agniveers will also receive an attractive 'Seva Nidhi' package on completion of their engagement period that includes contribution towards corpus fund by the Government along with interest accrued thereo

## batch embedding

In [16]:
ollama_client.embeddings?

Signature:
ollama_client.embeddings(
    model: str = '',
    prompt: str = '',
    options: Optional[ollama._types.Options] = None,
    keep_alive: Union[float, str, NoneType] = None,
) -> Mapping[str, Sequence[float]]
Docstring: <no docstring>
File:      /opt/conda/lib/python3.11/site-packages/ollama/_client.py
Type:      method

In [20]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.5 MB/s eta 0:00:0000:0100:01


In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

hi_text = "जीवन एक चॉकलेट बॉक्स की तरह है।"
chinese_text = "生活就像一盒巧克力。"

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# translate Hindi to French
tokenizer.src_lang = "hi"
encoded_hi = tokenizer(hi_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("fr"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "La vie est comme une boîte de chocolat."

# translate Chinese to English
tokenizer.src_lang = "zh"
encoded_zh = tokenizer(chinese_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Life is like a box of chocolate."

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['Life is like a box of chocolate.']

In [3]:
hi_text = "जीवन एक चॉकलेट बॉक्स की तरह है।"
tokenizer.src_lang = "hi"
encoded_hi = tokenizer(hi_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Life is like a chocolate box.']

In [7]:
%%timeit
hi_text = "Life is like a chocolate box."
tokenizer.src_lang = "en"
encoded_hi = tokenizer(hi_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("hi"))
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))

['जीवन एक चॉकलेट बॉक्स की तरह है।']
['जीवन एक चॉकलेट बॉक्स की तरह है।']
['जीवन एक चॉकलेट बॉक्स की तरह है।']
['जीवन एक चॉकलेट बॉक्स की तरह है।']
['जीवन एक चॉकलेट बॉक्स की तरह है।']
['जीवन एक चॉकलेट बॉक्स की तरह है।']
['जीवन एक चॉकलेट बॉक्स की तरह है।']
['जीवन एक चॉकलेट बॉक्स की तरह है।']
15.9 s ± 8.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['जीवन एक चॉकलेट बॉक्स की तरह है।']